In [10]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [11]:
#    [transforms.Resize(32),

transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 4

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
print(trainset.data.shape)
# classes = ('T-Shirt', 'Trouser', 'Pullover', 'Dress',
#            'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')


torch.Size([60000, 28, 28])


In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [13]:
def train_model(net, optimizer, criterion, epochs, trainloader):

  for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
      # if i <= 2000:
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # print(i)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

  print('Finished Training')



In [14]:
def predict_test(testloader):
  correct = 0
  total = 0
  # since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)
          # calculate outputs by running images through the network
          net.to(device)
          outputs = net(images)
          outputs = outputs.to(device)
          # the class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

### Use ADAM as Optimizer


In [15]:
net = Net().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer_adam = optim.Adam(net.parameters(), lr=0.0001)
train_model(net, optimizer_adam, criterion, 5, trainloader) #Change back to 5

predict_test(testloader)

[1,  2000] loss: 0.740
[1,  4000] loss: 0.254
[1,  6000] loss: 0.170
[1,  8000] loss: 0.139
[1, 10000] loss: 0.124
[1, 12000] loss: 0.119
[1, 14000] loss: 0.107
[2,  2000] loss: 0.080
[2,  4000] loss: 0.087
[2,  6000] loss: 0.076
[2,  8000] loss: 0.071
[2, 10000] loss: 0.070
[2, 12000] loss: 0.077
[2, 14000] loss: 0.078
[3,  2000] loss: 0.062
[3,  4000] loss: 0.056
[3,  6000] loss: 0.055
[3,  8000] loss: 0.063
[3, 10000] loss: 0.053
[3, 12000] loss: 0.057
[3, 14000] loss: 0.060
[4,  2000] loss: 0.041
[4,  4000] loss: 0.043
[4,  6000] loss: 0.055
[4,  8000] loss: 0.044
[4, 10000] loss: 0.046
[4, 12000] loss: 0.046
[4, 14000] loss: 0.055
[5,  2000] loss: 0.041
[5,  4000] loss: 0.042
[5,  6000] loss: 0.046
[5,  8000] loss: 0.040
[5, 10000] loss: 0.035
[5, 12000] loss: 0.047
[5, 14000] loss: 0.042
Finished Training
Accuracy of the network on the 10000 test images: 98 %


In [16]:
torch.save(net, 'net.pth')

In [17]:
from torchvision.transforms.transforms import Grayscale
from posixpath import split
transform = transforms.Compose(
    [transforms.Resize(28),
     transforms.Grayscale(num_output_channels=1),
     transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 4

trainsetSVHN = torchvision.datasets.SVHN('datasets/SVHN/train/', split='train',
                                        download=True, transform=transform)
trainloaderSVHN = torch.utils.data.DataLoader(trainsetSVHN, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testsetSVHN = torchvision.datasets.SVHN('datasets/SVHN/test/', split='test',
                                       download=True, transform=transform)
testloaderSVHN = torch.utils.data.DataLoader(testsetSVHN, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
print(trainsetSVHN.data.shape)
print(testsetSVHN.data.shape)

100%|██████████| 182040794/182040794 [00:04<00:00, 45060709.14it/s]


100%|██████████| 64275384/64275384 [00:02<00:00, 29490983.68it/s]


(73257, 3, 32, 32)
(26032, 3, 32, 32)


In [18]:
loaded_model = torch.load('net.pth')
loaded_model.to(device)
loaded_model.eval()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [19]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer_adam = optim.Adam(net.parameters(), lr=0.0001)
train_model(loaded_model, optimizer_adam, criterion, 5, trainloaderSVHN)

predict_test(testloaderSVHN)

[1,  2000] loss: 3.130
[1,  4000] loss: 3.072
[1,  6000] loss: 3.068
[1,  8000] loss: 3.067
[1, 10000] loss: 3.069
[1, 12000] loss: 3.089
[1, 14000] loss: 3.055
[1, 16000] loss: 3.056
[1, 18000] loss: 3.066
[2,  2000] loss: 3.042
[2,  4000] loss: 3.047
[2,  6000] loss: 3.071
[2,  8000] loss: 3.054
[2, 10000] loss: 3.107
[2, 12000] loss: 3.085
[2, 14000] loss: 3.051
[2, 16000] loss: 3.119
[2, 18000] loss: 3.104
[3,  2000] loss: 3.049
[3,  4000] loss: 3.060
[3,  6000] loss: 3.059
[3,  8000] loss: 3.085
[3, 10000] loss: 3.089
[3, 12000] loss: 3.111
[3, 14000] loss: 3.084
[3, 16000] loss: 3.076
[3, 18000] loss: 3.057
[4,  2000] loss: 3.078
[4,  4000] loss: 3.067
[4,  6000] loss: 3.059
[4,  8000] loss: 3.085
[4, 10000] loss: 3.087
[4, 12000] loss: 3.059
[4, 14000] loss: 3.075
[4, 16000] loss: 3.078
[4, 18000] loss: 3.071
[5,  2000] loss: 3.050
[5,  4000] loss: 3.071
[5,  6000] loss: 3.104
[5,  8000] loss: 3.076
[5, 10000] loss: 3.092
[5, 12000] loss: 3.049
[5, 14000] loss: 3.081
[5, 16000] 